In [1]:
import os
import ee
import geemap
import datetime
import requests
import numpy as np
from osgeo import gdal
from ipywidgets import HTML
import ipywidgets as widgets
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from ipyleaflet import WidgetControl
from scipy.signal import savgol_filter

In [2]:
# Start session in GEE 
ee.Authenticate()

project = 'ee-vl99956018'
ee.Initialize(project=project)


In [3]:
# Before fire date range
pre_start_date = ee.Date('2025-01-01')
pre_end_date = pre_start_date.advance(4, 'day')  # 2025-01-05

# After fire date range
pos_start_date = pre_end_date.advance(1, 'day')  # 2025-01-06
pos_end_date = pos_start_date.advance(10, 'day')  # 2025-01-16

# Coordinates of interest
latitude = 34.092615
longitude = -118.532875

#Region of interest (ROI) with a buffer of 15km
roi = ee.Geometry.Point(longitude, latitude).buffer(15000)


In [4]:
# Define palette color for visualization (RGB, NDVI and NBR)
# and color bar settings

rgb_vis = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3500,  
    'gamma': 1.2
}


ndvi_vis = {
    'min': -1,
    'max': 1,
    'palette': ['red', 'yellow', 'green']
}

nbr_vis = {
    'min': -1,
    'max': 1,
    'palette': ['white', 'black', 'red']
    
}

diff_nbr_vis = {
    'min': -0.5,
    'max': 0.5,
    'palette': ['blue', 'white', 'red']
}

#Initialize map with center and zoom
Map = geemap.Map(center=[latitude, longitude], zoom=13)


In [5]:
#define functions

def get_all_collections(start_date: str, end_date: str) -> dict:
    """
    Generates median Sentinel-2 composites (reflectance, NDVI, and NBR) for a specified date range.

    Args:
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.
        
    Returns:
        dict: Dictionary containing the image collection, NDVI, and NBR images.
    """
    try:

        collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
            .filterDate(start_date, end_date) \
            .filterBounds(roi) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

        # Select relevant bands and compute median
        image = collection.select('B8', 'B12', 'B4', 'B3', 'B2').median().clip(roi)

        # NDVI
        ndvi = collection.select(['B8', 'B4']) \
            .map(lambda img: img.normalizedDifference(['B8', 'B4']).rename('NDVI')) \
            .median().clip(roi)

        # NBR
        nbr = collection.select(['B8', 'B12']) \
            .map(lambda img: img.normalizedDifference(['B8', 'B12']).rename('NBR')) \
            .median().clip(roi)

        return {
            'image': image,
            'ndvi': ndvi,
            'nbr': nbr
        }
    except Exception as e:
        print(f"Error: {e}")
        

# Export function to Google Drive
def export_to_drive(image: ee.Image, image_title: str, folder_name='GEE_exports'):
    """
    Exports an image to Google Drive.
    
    Args:
        image (ee.Image): The image to export.
        image_title (str): The title for the exported image.
        folder_name (str): The name of the folder in Google Drive where the image will be saved.
    """
    try:
        task = ee.batch.Export.image.toDrive(
            image = image,
            description = image_title,
            folder = folder_name,
            fileNamePrefix = image_title.lower().replace(' ', '_'), #standardize file name
            scale = 10,  # Sentinel-2 resolution
            region = roi,
            fileFormat = 'GeoTIFF',
            maxPixels = 1e10
        )
        task.start()
        print(f'Started export task for {image_title}. Check your Google Drive folder "{folder_name}" when complete.')

    except Exception as e:
        print(f"Error: {e}")


# Create HTML labels for map output
def create_labels_html(legend_before: str, legend_after: str) -> tuple:
    """
    Creates two HTML label elements for map display.

    Args:
        legend_before (str): Text for the label positioned in the top-left corner.
        legend_after (str): Text for the label positioned in the top-right corner.

    Returns:
        tuple:
            A pair of IPython.display.HTML objects (before_label, after_label),
            each styled with absolute positioning and a white background for use
            in folium/geemap map overlays.
    """

    before_html = HTML(f'''<div style="position: absolute; top: 10px; left: 10px; z-index: 1000; 
                        background-color: white; padding: 5px; border-radius: 5px;
                        font-weight: bold; color: black;">{legend_before}</div>''')
                        
    after_html = HTML(f'''<div style="position: absolute; top: 70px; right: 10px; z-index: 1000; 
                         background-color: white; padding: 5px; border-radius: 5px;
                         font-weight: bold; color: black;">{legend_after}</div>''')

    return before_html, after_html


# Get the most recent date of available images
def get_most_recent_date(start_date: str) -> str:
    """
    Retrieves the most recent date of Sentinel-2 images in the specified date range.
    
    Args:
        start_date (str): Start date in 'YYYY-MM-DD' format.

    Returns:
        str: Most recent date in 'YYYY-MM-DD' format.    
    """
    try:
        # Today's date
        today = datetime.date.today().isoformat()

        # Create image collection
        collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
            .filterDate(start_date, today) \
            .filterBounds(roi) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
            .sort('system:time_start', False)  

        # Pick the most recent image
        most_recent_image = collection.first()

        # Get the date of the most recent image
        most_recent_date = ee.Date(most_recent_image.get('system:time_start')).format('YYYY-MM-dd')
        return most_recent_date.getInfo()

    except Exception as e:
        print(f"Error in get_most_recent_date: {e}")

In [6]:
# COMPUTE IMAGES FOR BEFORE AND AFTER FIRE IN RGB

map_rgb = Map

# Pre and post fire images in RGB
pre_result = get_all_collections(pre_start_date, pre_end_date)
pre_image = pre_result['image']

pos_result = get_all_collections(pos_start_date, pos_end_date)
pos_image = pos_result['image']


# Add split map to compare before and after fire images in RGB
left_layer = geemap.ee_tile_layer(pre_image, rgb_vis, 'Before Fire')
right_layer = geemap.ee_tile_layer(pos_image, rgb_vis, 'After Fire')

map_rgb.split_map(left_layer=left_layer, right_layer=right_layer)


# Add layers to the map (optional, can be toggled on/off in the layer control)
map_rgb.addLayer(pre_image, rgb_vis, 'Before Fire', False)
map_rgb.addLayer(pos_image, rgb_vis, 'After Fire', False)




# Add HTML labels to the map
before_html, after_html = create_labels_html('Before', 'After')
map_rgb.add_control(WidgetControl(widget=before_html, position='topleft'))
map_rgb.add_control(WidgetControl(widget=after_html, position='topright'))

map_rgb

# Export images to Google Drive (uncomment to use)
# export_to_drive(pre_image, "before_fire_RGB_image")
# export_to_drive(pos_image, "after_fire_RGB_image")




Map(center=[34.092615, -118.532875], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

In [19]:
import os
import numpy as np
import requests
from osgeo import gdal
import matplotlib.pyplot as plt

def download_image_as_pdf(
    image: ee.Image,
    image_title: str,
    scale=35,
    folder="results",
    roi=None,
    band_order=None,        # tuple of band indices (0-based) for R,G,B; None -> take first 3 bands
    vmin_pct=2,
    vmax_pct=98,
    gamma=0.8
) -> str:
    """
    Download an Earth Engine image as GeoTIFF, convert to a proper RGB image and save as PDF.

    - band_order: if your TIFF has many bands and you want a specific natural-color set,
      pass band_order=(r_idx,g_idx,b_idx) using 0-based indices. Example: (3,2,1) for Landsat 8 bands 4,3,2.
    - roi: region to pass to getDownloadURL if not global.
    """

    print(f"Downloading {image_title} as TIFF...")

    try:
        os.makedirs(folder, exist_ok=True)

        base_name = image_title.lower().replace(" ", "_")
        tif_path = os.path.join(folder, f"{base_name}.tif")
        pdf_path = os.path.join(folder, f"{base_name}.pdf")

        # Get URL (use roi if provided)
        download_args = {'scale': scale, 'format': 'GEO_TIFF'}
        if roi is not None:
            download_args['region'] = roi
        url = image.getDownloadURL(download_args)

        # Download TIFF
        response = requests.get(url, stream=True)
        response.raise_for_status()
        with open(tif_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

        print(f"TIFF saved: {tif_path}")

        # Read with GDAL
        ds = gdal.Open(tif_path)
        if ds is None:
            raise RuntimeError("GDAL failed to open the TIFF.")

        arr = ds.ReadAsArray()
        print("Raw TIFF array shape (bands?, y, x):", arr.shape)

        # Ensure shape is (y, x, bands)
        if arr.ndim == 3:
            # common GDAL shape is (bands, y, x)
            if arr.shape[0] <= 4 and arr.shape[0] != arr.shape[1]:
                arr = arr.transpose((1, 2, 0))  # -> (y, x, bands)
        elif arr.ndim == 2:
            # single band -> (y, x)
            pass
        else:
            raise ValueError(f"Unsupported array shape: {arr.shape}")

        # If single-band, apply a terrain colormap (or keep as stretched gray)
        if arr.ndim == 2:
            band = arr.astype("float32")
            p_low, p_high = np.nanpercentile(band, (vmin_pct, vmax_pct))
            band_s = np.clip((band - p_low) / (p_high - p_low + 1e-9), 0, 1)
            # Option: colorize with an "earth" style colormap
            cmap = plt.cm.terrain
            rgb = (cmap(band_s)[:, :, :3] * 255).astype("uint8")
        else:
            # arr is (y, x, bands)
            h, w, nb = arr.shape
            # Choose band order
            if band_order is None:
                # default: first three bands
                if nb >= 3:
                    band_order = (0, 1, 2)
                elif nb == 2:
                    band_order = (0, 0, 1)
                else:
                    band_order = (0, 0, 0)

            # Guard indices
            band_order = tuple(max(0, min(nb - 1, b)) for b in band_order)
            bands = [arr[:, :, i].astype("float32") for i in band_order]

            # If all bands are nearly identical (gray), try alternative orders commonly used:
            if nb >= 3:
                diffs = [np.nanstd(bands[i] - bands[j]) for i in range(3) for j in range(i+1,3)]
                # if small differences -> try common natural-color orders (Landsat/Sentinel)
                if max(diffs) < 1e-6:
                    alt_orders = [(3,2,1), (2,1,0), (4,3,2)]
                    for alt in alt_orders:
                        if all(0 <= a < nb for a in alt):
                            bands = [arr[:, :, a].astype("float32") for a in alt]
                            break

            # Per-band percentile stretch (vmin_pct–vmax_pct), then gamma
            rgb = np.zeros((h, w, 3), dtype="uint8")
            for i, b in enumerate(bands):
                p_low, p_high = np.nanpercentile(b, (vmin_pct, vmax_pct))
                b_s = np.clip((b - p_low) / (p_high - p_low + 1e-9), 0, 1)
                # gamma correction
                b_s = np.power(b_s, 1.0 / gamma)
                rgb[:, :, i] = (b_s * 255).astype("uint8")

        # Final sanity check
        if rgb.ndim != 3 or rgb.shape[2] != 3:
            raise RuntimeError("RGB image formation failed.")

        # Save as PDF with matplotlib (no axes)
        plt.figure(figsize=(10, 10))
        plt.imshow(rgb)
        plt.axis("off")
        plt.savefig(pdf_path, format="pdf", bbox_inches="tight", pad_inches=0)
        plt.close()

        print(f"PDF generated: {pdf_path}")
        return pdf_path

    except Exception as e:
        print(f"Error downloading {image_title} as PDF: {e}")
        return ""


In [20]:
download_image_as_pdf(pre_image, "before_fire_RGB_image") 
#download_image_as_pdf(pos_image, "after_fire_RGB_image")


TIFF saved: results/before_fire_rgb_image.tif
Raw TIFF array shape (bands?, y, x): (859, 1031)
PDF generated: results/before_fire_rgb_image.pdf


'results/before_fire_rgb_image.pdf'

In [9]:
# COMPUTE IMAGES FOR BEFORE AND AFTER FIRE IN NDVI

map_ndvi = Map


# Pre and post fire images in NDVI
pre_result = get_all_collections(pre_start_date, pre_end_date)
pre_image = pre_result['ndvi']

pos_result = get_all_collections(pos_start_date, pos_end_date)
pos_image = pos_result['ndvi']


# Add split map to compare before and after fire images in NDVI
left_layer = geemap.ee_tile_layer(pre_image, ndvi_vis, 'Before Fire')
right_layer = geemap.ee_tile_layer(pos_image, ndvi_vis, 'After Fire')

map_ndvi.split_map(left_layer=left_layer, right_layer=right_layer)


# Add layers to the map (optional, can be toggled on/off in the layer control)
map_ndvi.addLayer(pre_image, ndvi_vis, 'Before Fire', False)
map_ndvi.addLayer(pos_image, ndvi_vis, 'After Fire', False)


# NDVI color bar
map_ndvi.add_colorbar(
    vis_params=ndvi_vis,
    label='NDVI',
    layer_name='NDVI',
    orientation='vertical',  
    position='bottomright'
    )

# Add HTML labels to the map
before_html, adter_html = create_labels_html('Before', 'After')
map_ndvi.add_control(WidgetControl(widget=before_html, position='topleft'))
map_ndvi.add_control(WidgetControl(widget=adter_html, position='topright'))

map_ndvi

# Export images to Google Drive (uncomment to use)
# export_to_drive(pre_image, "before_fire_NDVI_image")
# export_to_drive(pos_image, "after_fire_NDVI_image")


Map(center=[34.092615, -118.532875], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

In [10]:
# COMPUTE IMAGES FOR BEFORE AND AFTER FIRE IN NBR

map_nbr = Map

# Pre and post fire images in NBR
pre_result = get_all_collections(pre_start_date, pre_end_date)
pre_image = pre_result['nbr']

pos_result = get_all_collections(pos_start_date, pos_end_date)
pos_image = pos_result['nbr']


# Calculate difference between pre and post fire images in NBR
dnbr = pre_image.subtract(pos_image)

# Define a threshold to identify burned areas
burned_mask = dnbr.gt(0.3)

# Calculate total burned area (in hec)
pixel_area = burned_mask.multiply(ee.Image.pixelArea()).divide(10000)  # m^2 to hec
area_burned = pixel_area.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=roi,
    scale=30,
    maxPixels=1e13
)

area_burned_dict = area_burned.getInfo()


# Calculate total area for context
total_area = ee.Image.pixelArea().divide(10000).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=roi,
    scale=30,
    maxPixels=1e13
)

# More robust way to handle the results
total_area_dict = total_area.getInfo()

# Extract the first (and usually only) value from each dictionary
burned_value = list(area_burned_dict.values())[0]
total_value = list(total_area_dict.values())[0]

print(f"Burned area (hectares): {round(burned_value, 2)}")
print(f"Total area (hectares): {round(total_value, 2)}")
print(f"Burned percentage: {round((burned_value / total_value) * 100, 2)}% of the ROI")

# Add split map to compare before and after fire images in NBR
left_layer = geemap.ee_tile_layer(pre_image, nbr_vis, 'Before Fire')
right_layer = geemap.ee_tile_layer(pos_image, nbr_vis, 'After Fire')


map_nbr.split_map(left_layer=left_layer, right_layer=right_layer)


# Add layers to the map (optional, can be toggled on/off in the layer control)
map_nbr.addLayer(pre_image, nbr_vis, 'Before Fire', False)
map_nbr.addLayer(pos_image, nbr_vis, 'After Fire', False)


# NBR color bar
map_nbr.add_colorbar(
    vis_params=nbr_vis,
    label='NBR',
    layer_name='NBR',
    orientation='vertical',
    position='bottomleft'
)

# Add HTML labels to the map
before_html, after_html = create_labels_html('Before', 'After')
map_nbr.add_control(WidgetControl(widget=before_html, position='topleft'))
map_nbr.add_control(WidgetControl(widget=after_html, position='topright'))



# Map for difference in NBR
Map_diff = geemap.Map(center=[latitude, longitude], zoom=13)
Map_diff.addLayer(dnbr, diff_nbr_vis, 'ΔNBR')

display(widgets.VBox([Map, Map_diff]))


# Export images to Google Drive (uncomment to use)
# export_to_drive(pre_image, "before_fire_NBR_image")
# export_to_drive(pos_image, "after_fire_NBR_image")
# export_to_drive(dnbr, "Difference NBR")




Burned area (hectares): 9357.08
Total area (hectares): 69822.28
Burned percentage: 13.4% of the ROI


In [11]:
def add_trace(fig, x: list, y: list, name: str, mode='lines+markers',
               line_color='blue', line_width = 2,
                 marker_color='red', marker_size = 8) -> None:
    
    """
    Helper function to add a trace to a Plotly figure.
    
    Args:
        fig (plotly.graph_objs._figure.Figure): The figure to add the trace to.
        x (list): The x-values of the trace.
        y (list): The y-values of the trace.
        name (str): The name of the trace.
        mode (str, optional): The mode of the trace. Default is 'lines+markers'.
        line_color (str, optional): The color of the line. Default is 'blue'.
        line_width (int, optional): The width of the line. Default is 2.
        marker_color (str, optional): The color of the markers. Default is 'red'.
        marker_size (int, optional): The size of the markers. Default is 8.
    Return:
        None
    """
    try:
        fig.add_trace(go.Scatter(
            x=x,
            y=y,
            mode=mode,
            name=name,
            line=dict(color=line_color, width=line_width),
            marker=dict(size=marker_size, color=marker_color),
        ))
    except Exception as e:
        print(f"Error adding trace for {name}: {e}")

def get_ndvi(image):
    """Adds an NDVI band (B8 - B4)/(B8 + B4) to the image."""
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

def get_nbr(image):
    """Adds an NBR band (B12 - B8)/(B12 + B8) to the image."""
    nbr = image.normalizedDifference(['B12', 'B8']).rename('NBR')
    return image.addBands(nbr)

def ndvi_nbr_plot(start_date:str, end_date:str) -> None:
    """
    Plots NDVI and NBR time series for the global ROI and a given date range,
    applying Savitzky–Golay smoothing, and saves the result as an interactive HTML file.

    Args:
        start_date (str): The start date for the image collection filter in 'YYYY-MM-DD' format.
        end_date (str): The end date for the image collection filter in 'YYYY-MM-DD' format.

    Returns:
        None
    """
    try:
        # Load Sentinel-2 SR collection
        collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                        .filterDate(start_date, end_date)
                        .filterBounds(roi)
                        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 40))
                        .map(get_ndvi)
                        .map(get_nbr)
                        .select(['NDVI', 'NBR']))
        

        # Extract NDVI and NBR means and date as Feature
        def reduce_img(img):
            stats = img.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=roi,
                scale=10,
                maxPixels=1e9
            )
            return ee.Feature(None, {
                'date': img.date().format('YYYY-MM-dd'),
                'NDVI': stats.get('NDVI'),
                'NBR': stats.get('NBR')
            })


        features = collection.map(reduce_img)


        stats_list = features.aggregate_array('date').getInfo()
        ndvi_values = features.aggregate_array('NDVI').getInfo()
        nbr_values = features.aggregate_array('NBR').getInfo()

        # Arguments to smooth curves
        window = 19     # must be odd, increase to smooth more
        poly = 6       # polynomial order

        # Smooth curves
        ndvi_smooth = savgol_filter(ndvi_values, window_length=window, polyorder=poly)
        nbr_smooth  = savgol_filter(nbr_values,  window_length=window, polyorder=poly)


        # Convert to datetime
        dates = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in stats_list]

        # Plot NDVI and NBR curves in the same figure
        fig = go.Figure()

        add_trace(fig, dates, ndvi_values, 'NDVI', line_color='green', marker_color='green')
        add_trace(fig, dates, ndvi_smooth, 'NDVI (smoothed)', line_color='green', marker_color='green', mode = 'lines')
        
        add_trace(fig, dates, nbr_values, 'NBR', line_color='red', marker_color='red')
        add_trace(fig, dates, nbr_smooth, 'NBR (smoothed)', line_color='red', marker_color='red', mode= "lines")


        # Add vertical line at 2025-01-07
        fig.add_vline(
            x=datetime.datetime(2025, 1, 7).timestamp() * 1000,  # Convert to milliseconds since epoch
            line_width=2,
            line_dash="dash",
            line_color="blue",
            annotation_text="First Fire detected",
            annotation_position="top right"
        )

        fig.update_layout(
            title='NDVI and NBR TimeSeries',
            xaxis_title='Date',
            yaxis_title='Index Value',
            xaxis=dict(showgrid=True),
            yaxis=dict(showgrid=True, title='Index Value'),
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1
            ),
            template='plotly_white',
            hovermode='x unified'
        )

        # Save plot
        os.makedirs("results", exist_ok=True)
        fig.write_html("results/ndvi_nbr_plot.html")
        print("Graphic saved at 'results/ndvi_nbr_plot.html'")
        fig.show(renderer = "browser")

    except Exception as e:
        print(f"Error in ndvi_nbr_plot: {e}")


In [12]:
curve_start_date = '2024-10-01'
curve_end_date = '2025-04-01'

# Run the function
ndvi_nbr_plot(curve_start_date, curve_end_date)

Graphic saved at 'results/ndvi_nbr_plot.html'
Opening in existing browser session.


In [13]:
# GET IMAGES AFTER FIRE AND TODAY'S DATE

# After images
pos_result = get_all_collections(pos_start_date, pos_end_date)
pos_image = pos_result['image']

# Get most recent date
recent_date = get_most_recent_date('2025-04-01')
today = datetime.date.today().isoformat()

# Today images
today_result = get_all_collections(recent_date, today)
today_image = today_result['image']

print(f"The most recent date is {recent_date} for a cloud coverage of 10% or less.")

left_layer = geemap.ee_tile_layer(pos_image, rgb_vis, 'After Fire')
right_layer = geemap.ee_tile_layer(today_image, rgb_vis, 'Today')

Map.split_map(left_layer=left_layer, right_layer=right_layer)

# Add layers  
Map.addLayer(pos_image, rgb_vis, 'After Fire', False)
Map.addLayer(today_image, rgb_vis, 'Today', False)

# export_to_drive(pre_image, "before_fire_RGB_image")
# export_to_drive(pos_image, "after_fire_RGB_image")


# Add map control
antes_html, depois_html = create_labels_html('After', 'Today')
Map.add_control(WidgetControl(widget=antes_html, position='topleft'))
Map.add_control(WidgetControl(widget=depois_html, position='topright'))

Map

The most recent date is 2025-11-10 for a cloud coverage of 10% or less.


Map(bottom=837395.0, center=[34.092615, -118.532875], controls=(ZoomControl(options=['position', 'zoom_in_text…